### SVM 

In [ ]:
import numpy as np
import scipy.io
from scipy.io import wavfile
import matplotlib.pyplot as plt
from sklearn import mixture, svm, ensemble, covariance
import os

datafolder = 'D:/WORK/dataset/'    
defective = ('9009141939', '9014404791', '9025346235')
folders = os.listdir(datafolder)
[folders.remove(defective[i]) for i in range(len(defective))] #удаляем из списка бракованные папки

gamma_variable = [0.025] #np.arange(0,1,0.01) #0.025; от 0.01 до 0.05, переобучение при приближении к 1
nu_variable = [0.1] #np.arange(0,1,0.01) #0.2; от 0.1 до 0.07; недообучение при уменьшении, переобучение при увеличении
threshold = 0.07
number_of_components = 64

def most_common(lst):
    return max(set(lst), key=lst.count)


def create_supervector(w, m, v):
    """
    Returns Gaussian Supervector (GSV) using bounded Kullback-Leibler
    distance method
    """

    #Verify input conforms to proper shapes for matrix math
    assert np.size(w,0) == np.size(m,0) == np.size(v,0)
    assert np.size(m,1) == np.size(v,1) == np.size(v,2)

    num_mixtures = np.size(w,0)
    num_mfccs = np.size(m,1)

    for i in range(num_mixtures):
        m[i,:] = np.sqrt(w[i]) * m[i,:] * np.diag(v[i,:,:]**(-.5))
    
    return np.reshape(m,(num_mixtures*num_mfccs,1))



for g, n in zip(gamma_variable, nu_variable):
    
    quality = [] 
    quality_imposter = []

    clf =  svm.OneClassSVM(kernel='rbf', degree=3, gamma= float(g), coef0=0.0, 
                    tol=0.001, nu= float(n), shrinking=True, cache_size=200, 
                    verbose=False, max_iter=-1, random_state=None)
    
    
    print( "gamma = " + str(g), ", nu = " + str(n))
                           
    for i in range(0, 100): 


        """
        Cобираем данные - обучающая выборка, тестовая, imposter model
        """
        try:
            speaker_train = mfcc_mat[i][0][0][0]
            speaker_test =  mfcc_mat[i][0][0][1][0]
            imposter = imposter_mat



            """
            Обучаем на всех типах данных GMM 
            """

            X_train = speaker_train
            #GMM = mixture.GaussianMixture(n_components = number_of_components, max_iter=3000).fit(speaker_train)
            #X_train = create_supervector(GMM.weights_, GMM.means_, GMM.covariances_ )
            #X_train = GMM.means_ 
            clf.fit(X_train)


            X_test = speaker_test
            #X_test = []
            #for sample in range(0, len(speaker_test)):
                #GMM = mixture.GaussianMixture(n_components = number_of_components,  max_iter=3000).fit(speaker_test[sample])
                #supervector = create_supervector(GMM.weights_, GMM.means_, GMM.covariances_ )
                #X_test.append(supervector)
                #X_test.append(GMM.means_)



            X_imposter = imposter 
            #X_imposter = []
            #for sample in range(0, len(imposter)):
                #GMM = mixture.GaussianMixture(n_components = number_of_components,  max_iter=3000).fit(imposter[sample])
                #supervector = create_supervector(GMM.weights_, GMM.means_, GMM.covariances_ )
                #X_imposter.append(supervector)
                #X_imposter.append(GMM.means_)


            """
            Получаем ответы для тестовой выборки: 
            predict содержит +1/-1 для каждой записи из теста
            quality показывает, сколько правильных ответов дал алгоритм 
            """
            predictions_test = [clf.predict(X_test[i]) for i in range(len(X_test))]
            predict_test = [predictions_test[i].tolist().count(1)/len(predictions_test[i].tolist()) for i in range(len(predictions_test))]
            print(predict_test)
            quality.append(predict_test)


            """
            Получаем ответы для imposter model
            """
            predictions_imposter = [clf.predict(X_imposter[i]) for i in range(len(X_imposter))]
            predict_imposter = [predictions_imposter[i].tolist().count(1)/len(predictions_imposter[i].tolist()) for i in range( len(predictions_imposter) )]
            print(predict_imposter)
            quality_imposter.append(predict_imposter)


            print(folders[i], ' : ', len([predict_test[i] for i in range(len(predict_test)) if predict_test[i] >= threshold])/len(predict_test), 
                  ', imposter: ', len([predict_imposter[i] for i in range(len(predict_imposter)) if predict_imposter[i] <= threshold])/len(predict_imposter), '\n')


        except: 
            pass  
        
    """
    ACCURACY 
    """

    flag = [item for sublist in quality for item in sublist]
    flag_imposter = [item for sublist in quality_imposter for item in sublist]
    print(len([flag[i] for i in range(len(flag)) if flag[i] >= threshold])/len(flag))
    print(len([flag_imposter[i] for i in range(len(flag_imposter)) if flag_imposter[i] < threshold])/len(flag_imposter))
    print('\n')

In [76]:
threshold = 0.2

print(len([flag[i] for i in range(len(flag)) if flag[i] >= threshold])/len(flag))
print(len([flag_imposter[i] for i in range(len(flag_imposter)) if flag_imposter[i] < threshold])/len(flag_imposter))

0.9520433694745621
0.953405017921147


In [35]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

0.09303642196640875
0.02216216216216216
